In [22]:
#Proceso de Decisión de Markov (MDP)

import numpy as np
import random

estados = ['A', 'B', 'C']
acciones = ['izquierda', 'abajo']
recompensas = np.random.randint(0,10,size=(len(estados),len(acciones)))
print('recompensas',recompensas)

#Función de transición aleatoria
def transicion_aleatoria():
    return np.random.choice(estados)

#Generación de datos
estado_actual = np.random.choice(estados)
accion = np.random.choice(acciones)
nuevo_estado = transicion_aleatoria()
recompensa = recompensas[estados.index(estado_actual), acciones.index(accion)]

print("Estado actual:", estado_actual)
print("Acción tomada:", accion)
print("Nuevo estado:", nuevo_estado)
print("Recompensa:", recompensa)


recompensas [[5 1]
 [5 4]
 [8 6]]
Estado actual: A
Acción tomada: izquierda
Nuevo estado: B
Recompensa: 5


In [23]:
# Definición de transiciones aleatorias
def generar_transiciones(estados, acciones):
    transiciones = {}
    for estado in estados:
        transiciones[estado] = {}
        for accion in acciones:
            probabilidades = np.random.rand(len(estados))  # Probabilidades aleatorias
            probabilidades /= probabilidades.sum()  # Normalizar para que sumen 1
            transiciones[estado][accion] = {nuevo_estado: prob for nuevo_estado, prob in zip(estados, probabilidades)}
    return transiciones

transiciones = generar_transiciones(estados, acciones)
print(transiciones)

{'A': {'izquierda': {'A': 0.5013102184008875, 'B': 0.4209757920515474, 'C': 0.07771398954756498}, 'abajo': {'A': 0.005565797773062945, 'B': 0.25297855996959245, 'C': 0.7414556422573446}}, 'B': {'izquierda': {'A': 0.41997893463477626, 'B': 0.42894118076107757, 'C': 0.15107988460414606}, 'abajo': {'A': 0.3496211726751686, 'B': 0.30812943111488506, 'C': 0.34224939620994627}}, 'C': {'izquierda': {'A': 0.25146703195456155, 'B': 0.21287666913456385, 'C': 0.5356562989108745}, 'abajo': {'A': 0.3597205922146921, 'B': 0.3575041281810375, 'C': 0.2827752796042703}}}


In [24]:
# Clase MDP
class MDP:
    def __init__(self, estados, acciones, transiciones, recompensas):
        self.estados = estados
        self.acciones = acciones
        self.transiciones = transiciones
        self.recompensas = recompensas

# Crear la instancia del MDP
mdp = MDP(estados, acciones, transiciones, recompensas)

def calcular_valor_estado(mdp, gamma=0.9, theta=0.01):
    valores = {estado: 0 for estado in mdp.estados}
    while True:
        delta = 0
        for estado in mdp.estados:
            valor_previo = valores[estado]
            valor_nuevo = 0
            for accion in mdp.acciones:
                if estado in mdp.transiciones and accion in mdp.transiciones[estado]:
                    for nuevo_estado in mdp.transiciones[estado][accion]:  # Iterar solo sobre los estados posibles
                        prob_transicion = mdp.transiciones[estado][accion].get(nuevo_estado, 0)
                        recompensa = mdp.recompensas[mdp.estados.index(estado)][mdp.acciones.index(accion)]
                        valor_nuevo += prob_transicion * (recompensa + gamma * valores[nuevo_estado])
            valores[estado] = np.clip(valor_nuevo, -1e10, 1e10)  # Limitar el rango de valores
            delta = max(delta, abs(valor_previo - valores[estado]))
        if delta < theta:
            break
    return valores

#Ejemplos de uso
valores_estados = calcular_valor_estado(mdp)
print('Valores de los estados:', valores_estados)

Valores de los estados: {'A': 10000000000.0, 'B': 10000000000.0, 'C': 10000000000.0}


In [25]:
#Propiedades de Markov
#Escribe una función para verificar si un MDP dado cumple con la propiedad de Markov
def verificar_propiedad_markov(mdp):
    for estado in mdp.estados:
        for accion in mdp.acciones:
            suma_probabilidades = sum(mdp.transiciones[estado][accion].values())
            if not np.isclose(suma_probabilidades, 1.0):
                return False
        return True

#Ejemplo de uso
print('Cumple con la propiedad de Markov:', verificar_propiedad_markov(mdp))


Cumple con la propiedad de Markov: True


In [26]:
print(type(mdp.recompensas))
print(mdp.recompensas)
print(mdp.transiciones)

<class 'numpy.ndarray'>
[[5 1]
 [5 4]
 [8 6]]
{'A': {'izquierda': {'A': 0.5013102184008875, 'B': 0.4209757920515474, 'C': 0.07771398954756498}, 'abajo': {'A': 0.005565797773062945, 'B': 0.25297855996959245, 'C': 0.7414556422573446}}, 'B': {'izquierda': {'A': 0.41997893463477626, 'B': 0.42894118076107757, 'C': 0.15107988460414606}, 'abajo': {'A': 0.3496211726751686, 'B': 0.30812943111488506, 'C': 0.34224939620994627}}, 'C': {'izquierda': {'A': 0.25146703195456155, 'B': 0.21287666913456385, 'C': 0.5356562989108745}, 'abajo': {'A': 0.3597205922146921, 'B': 0.3575041281810375, 'C': 0.2827752796042703}}}


In [27]:
#Propiedades de recompensa
#Escribe una función para calcular la
#recompensa promedio por acción en un MDP.

import numpy as np

def calcular_recompensa_promedio(mdp):
    recompensa_total = 0
    total_acciones = 0
    for estado in mdp.estados:
        for accion in mdp.acciones:
            for nuevo_estado in mdp.transiciones[estado][accion]:
                indice_estado = mdp.estados.index(estado)
                indice_accion = mdp.acciones.index(accion)
                recompensa_total += mdp.recompensas[indice_estado, indice_accion]
                total_acciones += 1
    return recompensa_total / total_acciones if total_acciones > 0 else 0

# Ejemplo de uso
print('Recompensa promedio por acción:', calcular_recompensa_promedio(mdp))

Recompensa promedio por acción: 4.833333333333333
